In [28]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from math import *

%matplotlib inline

In [29]:
np.random.seed(1)
x = np.random.random(60)
y = np.random.random(60)

In [30]:
signal_exists = ['exist','no_exist']
signal_moves = ['BUYOPEN','SELLOPEN','CLOSESHORT','CLOSELONG']

In [31]:
def generate_signal(priceA, priceB):
    '''Find the trading signal in one window, default as 60 days'''
    
    # test the existence of signal
    tfA = len(np.unique(np.isnan(priceA)))
    tfB = len(np.unique(np.isnan(priceB)))
            
    if  tfA == 2 or tfB == 2:
        signal_exist = signal_exists[0]
    else:
        signal_exist = signal_exists[1]

        # calculate return for two stocks
        Areturn = np.diff(priceA)/priceA[:(len(priceA)-1)]
        Breturn = np.diff(priceB)/priceB[:(len(priceB)-1)]

        # regression on return
        beta, beta0, r_value, p_value, std_err = stats.linregress(Areturn, Breturn)

        # get the residual epsilon_t
        e_t = np.array(Breturn - beta0 - beta*Areturn)

        # auxiliary process X_t
        Xt = []
        for i in range(len(Areturn)):
            Xt.append(np.sum(e_t[:i+1]))

        # regression on X_t
        length = len(Xt)
        Xt_vec = np.array(Xt)
        b, a, r_value_x, p_value_x, std_err_x = stats.linregress(Xt_vec[1:],Xt_vec[:length-1])

        # get the residual zeta_t
        z_t = Xt_vec[:length-1] - a - b*Xt_vec[1:]
        var_z = np.var(z_t)

        # calculate the s-score
        s_score = -a*sqrt(1-b**2)/((1-b)*sqrt(var_z)) + int(a/(1-b))*sqrt((1-b**2)/var_z)

        # trading signal
        if s_score < -1.25:
            signal_move = signal_moves[0]
        elif s_score > 1.25:
            signal_move = signal_moves[1]
        elif s_score < 0.75:
            signal_move = signal_moves[2]
        elif s_score > -0.5:
            signal_move = signal_moves[3]
        else:
            print('warning!')

        return beta,signal_move,signal_exist

In [32]:
generate_signal(x,y)

(-0.0010757644051472652, 'CLOSESHORT', 'no_exist')